In [1]:
# Crypto MACD Crossover Strategy Backtesting System
# Description: Implements a trend following MACD crossover strategy for cryptocurrency trading

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timezone, timedelta
import pytz
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set paths
DATA_DIR = Path("data")
OUTPUT_DIR = Path("output")
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Data Loader class
class DataLoader:
    def __init__(self, symbol, timeframe="10m"):
        """
        Initialize the DataLoader with a symbol and timeframe
        
        Parameters:
        -----------
        symbol : str
            Trading symbol (e.g., 'btcusd', 'ethusd')
        timeframe : str
            Timeframe of the data (e.g., '10m')
        """
        self.symbol = symbol.lower()
        self.timeframe = timeframe
        self.data = None
        
    def load_data(self):
        """
        Load data from CSV file
        
        Returns:
        --------
        pd.DataFrame
            DataFrame with loaded and preprocessed data
        """
        file_path = DATA_DIR / f"{self.symbol}_{self.timeframe}.csv"
        
        # Check if file exists
        if not file_path.exists():
            raise FileNotFoundError(f"Data file not found: {file_path}")
        
        # Load data
        df = pd.read_csv(file_path)
        
        # Convert time columns to datetime
        df['time_utc'] = pd.to_datetime(df['time_utc'])
        df['time_est'] = pd.to_datetime(df['time_est'])
        
        # Set time_utc as index
        df.set_index('time_utc', inplace=True)
        
        # Check for missing intervals
        time_diff = df.index.to_series().diff().dt.total_seconds() / 60
        if not all(time_diff.dropna() == 10):
            print(f"Warning: Data for {self.symbol} has missing or irregular intervals")
            
        # Check for duplicates
        if df.index.duplicated().any():
            print(f"Warning: Duplicate timestamps found in {self.symbol} data")
            df = df[~df.index.duplicated()]
        
        # Check for null values
        if df[['o', 'h', 'l', 'c', 'v']].isnull().any().any():
            print(f"Warning: Null values found in {self.symbol} OHLCV data")
            # Fill missing values using forward fill
            df[['o', 'h', 'l', 'c', 'v']] = df[['o', 'h', 'l', 'c', 'v']].fillna(method='ffill')
        
        # Ensure regular 10-minute intervals
        start_time = df.index.min()
        end_time = df.index.max()
        full_range = pd.date_range(start=start_time, end=end_time, freq='10min')
        
        # Reindex and interpolate if needed
        if len(full_range) != len(df):
            df = df.reindex(full_range)
            df[['o', 'h', 'l', 'c', 'v']] = df[['o', 'h', 'l', 'c', 'v']].interpolate(method='linear')
            print(f"Reindexed {self.symbol} data to ensure regular 10-minute intervals")
        
        self.data = df
        return df
    
    def get_ohlcv(self):
        """
        Get OHLCV data
        
        Returns:
        --------
        pd.DataFrame
            DataFrame with OHLCV data
        """
        if self.data is None:
            self.load_data()
        
        return self.data[['o', 'h', 'l', 'c', 'v']]

# Strategy class for MACD crossover
class MACDStrategy:
    def __init__(self, short_window=12, long_window=26, signal_window=9):
        """
        Initialize MACD strategy with parameters
        
        Parameters:
        -----------
        short_window : int
            Short EMA window
        long_window : int
            Long EMA window
        signal_window : int
            Signal EMA window
        """
        self.short_window = short_window
        self.long_window = long_window
        self.signal_window = signal_window
        
    def generate_signals(self, df):
        """
        Generate MACD crossover signals
        
        Parameters:
        -----------
        df : pd.DataFrame
            DataFrame with OHLCV data
            
        Returns:
        --------
        pd.DataFrame
            DataFrame with MACD signals
        """
        # Make a copy of the dataframe
        data = df.copy()
        
        # Calculate short and long EMAs
        data['ema_short'] = data['c'].ewm(span=self.short_window, adjust=False).mean()
        data['ema_long'] = data['c'].ewm(span=self.long_window, adjust=False).mean()
        
        # Calculate MACD and signal line
        data['macd'] = data['ema_short'] - data['ema_long']
        data['signal_line'] = data['macd'].ewm(span=self.signal_window, adjust=False).mean()
        
        # Calculate histogram (MACD - Signal)
        data['histogram'] = data['macd'] - data['signal_line']
        
        # Generate signals
        data['signal'] = 0
        
        # Crossover up (MACD crosses above signal line)
        data.loc[data['macd'] > data['signal_line'], 'signal'] = 1
        
        # Crossover down (MACD crosses below signal line)
        data.loc[data['macd'] < data['signal_line'], 'signal'] = -1
        
        # Generate actual trading signals (to account for the requirement that signals are generated on close but executed on next candle's open)
        data['position'] = data['signal'].shift(1)
        data['position'] = data['position'].fillna(0)
        
        return data

# Backtesting Engine class
class BacktestEngine:
    def __init__(self, initial_capital=10000, position_size=1.0, 
                 take_profit=None, stop_loss=None, trailing_stop=None):
        """
        Initialize Backtest Engine
        
        Parameters:
        -----------
        initial_capital : float
            Initial capital for the backtest
        position_size : float
            Position size as a percentage of capital (0-1)
        take_profit : float or None
            Take profit level as a percentage (e.g., 0.05 for 5%)
        stop_loss : float or None
            Stop loss level as a percentage (e.g., 0.03 for 3%)
        trailing_stop : float or None
            Trailing stop as a percentage (e.g., 0.02 for 2%)
        """
        self.initial_capital = initial_capital
        self.position_size = position_size
        self.take_profit = take_profit
        self.stop_loss = stop_loss
        self.trailing_stop = trailing_stop
        
    def run(self, data):
        """
        Run backtest on the data
        
        Parameters:
        -----------
        data : pd.DataFrame
            DataFrame with signals
            
        Returns:
        --------
        pd.DataFrame
            DataFrame with backtest results
        """
        # Make a copy of the dataframe
        backtest_data = data.copy()
        
        # Initialize portfolio metrics
        backtest_data['capital'] = self.initial_capital
        backtest_data['holdings'] = 0
        backtest_data['cash'] = self.initial_capital
        
        # Trading state
        in_position = False
        entry_price = 0
        entry_time = None
        exit_price = 0
        exit_time = None
        stop_price = 0
        take_profit_price = 0
        current_position = 0
        
        # Trade log
        trades = []
        
        # Run backtest
        for i in range(1, len(backtest_data)):
            current_row = backtest_data.iloc[i]
            prev_row = backtest_data.iloc[i-1]
            
            # Default to previous values
            backtest_data.loc[backtest_data.index[i], 'holdings'] = prev_row['holdings']
            backtest_data.loc[backtest_data.index[i], 'cash'] = prev_row['cash']
            
            # Check for exit conditions if in position
            if in_position:
                # Update trailing stop if applicable
                if self.trailing_stop is not None:
                    if current_position > 0:  # Long position
                        # Update stop if price moves in our favor
                        if current_row['h'] > entry_price:
                            new_stop = current_row['h'] * (1 - self.trailing_stop)
                            if new_stop > stop_price:
                                stop_price = new_stop
                    else:  # Short position
                        # Update stop if price moves in our favor
                        if current_row['l'] < entry_price:
                            new_stop = current_row['l'] * (1 + self.trailing_stop)
                            if new_stop < stop_price or stop_price == 0:
                                stop_price = new_stop
                
                # Check stop loss (for both fixed and trailing)
                if self.stop_loss is not None or self.trailing_stop is not None:
                    if current_position > 0:  # Long position
                        if current_row['l'] <= stop_price:
                            # Exit at stop price
                            exit_price = stop_price
                            exit_time = backtest_data.index[i]
                            cash_change = current_position * exit_price
                            backtest_data.loc[backtest_data.index[i], 'cash'] += cash_change
                            backtest_data.loc[backtest_data.index[i], 'holdings'] = 0
                            in_position = False
                            
                            # Log trade
                            trades.append({
                                'entry_time': entry_time,
                                'entry_price': entry_price,
                                'exit_time': exit_time,
                                'exit_price': exit_price,
                                'position': current_position,
                                'pnl': (exit_price - entry_price) * current_position,
                                'pnl_pct': (exit_price - entry_price) / entry_price,
                                'exit_reason': 'stop_loss'
                            })
                            
                            current_position = 0
                            continue
                    else:  # Short position
                        if current_row['h'] >= stop_price:
                            # Exit at stop price
                            exit_price = stop_price
                            exit_time = backtest_data.index[i]
                            cash_change = -current_position * exit_price
                            backtest_data.loc[backtest_data.index[i], 'cash'] += cash_change
                            backtest_data.loc[backtest_data.index[i], 'holdings'] = 0
                            in_position = False
                            
                            # Log trade
                            trades.append({
                                'entry_time': entry_time,
                                'entry_price': entry_price,
                                'exit_time': exit_time,
                                'exit_price': exit_price,
                                'position': current_position,
                                'pnl': (entry_price - exit_price) * -current_position,
                                'pnl_pct': (entry_price - exit_price) / entry_price,
                                'exit_reason': 'stop_loss'
                            })
                            
                            current_position = 0
                            continue
                
                # Check take profit
                if self.take_profit is not None:
                    if current_position > 0:  # Long position
                        if current_row['h'] >= take_profit_price:
                            # Exit at take profit price
                            exit_price = take_profit_price
                            exit_time = backtest_data.index[i]
                            cash_change = current_position * exit_price
                            backtest_data.loc[backtest_data.index[i], 'cash'] += cash_change
                            backtest_data.loc[backtest_data.index[i], 'holdings'] = 0
                            in_position = False
                            
                            # Log trade
                            trades.append({
                                'entry_time': entry_time,
                                'entry_price': entry_price,
                                'exit_time': exit_time,
                                'exit_price': exit_price,
                                'position': current_position,
                                'pnl': (exit_price - entry_price) * current_position,
                                'pnl_pct': (exit_price - entry_price) / entry_price,
                                'exit_reason': 'take_profit'
                            })
                            
                            current_position = 0
                            continue
                    else:  # Short position
                        if current_row['l'] <= take_profit_price:
                            # Exit at take profit price
                            exit_price = take_profit_price
                            exit_time = backtest_data.index[i]
                            cash_change = -current_position * exit_price
                            backtest_data.loc[backtest_data.index[i], 'cash'] += cash_change
                            backtest_data.loc[backtest_data.index[i], 'holdings'] = 0
                            in_position = False
                            
                            # Log trade
                            trades.append({
                                'entry_time': entry_time,
                                'entry_price': entry_price,
                                'exit_time': exit_time,
                                'exit_price': exit_price,
                                'position': current_position,
                                'pnl': (entry_price - exit_price) * -current_position,
                                'pnl_pct': (entry_price - exit_price) / entry_price,
                                'exit_reason': 'take_profit'
                            })
                            
                            current_position = 0
                            continue
            
            # Check for position changes based on MACD signals
            signal = current_row['position']
            
            if signal != 0:
                # If in position and signal is opposite, close position first
                if in_position and ((current_position > 0 and signal < 0) or (current_position < 0 and signal > 0)):
                    # Exit at current open price
                    exit_price = current_row['o']
                    exit_time = backtest_data.index[i]
                    
                    if current_position > 0:  # Long position
                        cash_change = current_position * exit_price
                    else:  # Short position
                        cash_change = -current_position * exit_price
                    
                    backtest_data.loc[backtest_data.index[i], 'cash'] += cash_change
                    backtest_data.loc[backtest_data.index[i], 'holdings'] = 0
                    
                    # Log trade
                    trades.append({
                        'entry_time': entry_time,
                        'entry_price': entry_price,
                        'exit_time': exit_time,
                        'exit_price': exit_price,
                        'position': current_position,
                        'pnl': (exit_price - entry_price) * current_position if current_position > 0 else (entry_price - exit_price) * -current_position,
                        'pnl_pct': (exit_price - entry_price) / entry_price if current_position > 0 else (entry_price - exit_price) / entry_price,
                        'exit_reason': 'signal'
                    })
                    
                    in_position = False
                    current_position = 0
                
                # Enter new position if not in position or just exited
                if not in_position:
                    # Entry price is the current open price
                    entry_price = current_row['o']
                    entry_time = backtest_data.index[i]
                    
                    # Calculate position size based on capital
                    position_value = prev_row['cash'] * self.position_size
                    
                    if signal > 0:  # Buy signal
                        # Enter long position
                        current_position = position_value / entry_price
                        cash_change = -position_value
                    else:  # Sell signal
                        # Enter short position
                        current_position = -position_value / entry_price
                        cash_change = position_value
                    
                    # Update portfolio
                    backtest_data.loc[backtest_data.index[i], 'cash'] += cash_change
                    backtest_data.loc[backtest_data.index[i], 'holdings'] = current_position * current_row['c']
                    
                    # Set stop loss and take profit levels
                    if self.stop_loss is not None:
                        if current_position > 0:  # Long position
                            stop_price = entry_price * (1 - self.stop_loss)
                        else:  # Short position
                            stop_price = entry_price * (1 + self.stop_loss)
                    elif self.trailing_stop is not None:
                        if current_position > 0:  # Long position
                            stop_price = entry_price * (1 - self.trailing_stop)
                        else:  # Short position
                            stop_price = entry_price * (1 + self.trailing_stop)
                    
                    if self.take_profit is not None:
                        if current_position > 0:  # Long position
                            take_profit_price = entry_price * (1 + self.take_profit)
                        else:  # Short position
                            take_profit_price = entry_price * (1 - self.take_profit)
                    
                    in_position = True
            
            # Update portfolio value
            if in_position:
                backtest_data.loc[backtest_data.index[i], 'holdings'] = current_position * current_row['c']
            
            backtest_data.loc[backtest_data.index[i], 'capital'] = backtest_data.loc[backtest_data.index[i], 'cash'] + backtest_data.loc[backtest_data.index[i], 'holdings']
        
        # Create trade log DataFrame
        trade_log = pd.DataFrame(trades) if trades else pd.DataFrame()
        
        return backtest_data, trade_log

# Visualization functions
def plot_price_with_signals(data, symbol, output_dir):
    """
    Plot price chart with buy/sell signals
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame with backtest results
    symbol : str
        Trading symbol
    output_dir : Path
        Directory to save the plot
    """
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                         vertical_spacing=0.05, 
                         subplot_titles=(f'{symbol.upper()} Price with Signals', 'MACD'))
    
    # Price chart
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['o'],
            high=data['h'],
            low=data['l'],
            close=data['c'],
            name='Price'
        ),
        row=1, col=1
    )
    
    # Add buy signals
    buy_signals = data[data['position'] == 1]
    fig.add_trace(
        go.Scatter(
            x=buy_signals.index,
            y=buy_signals['o'],
            mode='markers',
            marker=dict(color='green', size=10, symbol='triangle-up'),
            name='Buy Signal'
        ),
        row=1, col=1
    )
    
    # Add sell signals
    sell_signals = data[data['position'] == -1]
    fig.add_trace(
        go.Scatter(
            x=sell_signals.index,
            y=sell_signals['o'],
            mode='markers',
            marker=dict(color='red', size=10, symbol='triangle-down'),
            name='Sell Signal'
        ),
        row=1, col=1
    )
    
    # MACD
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data['macd'],
            mode='lines',
            line=dict(color='blue'),
            name='MACD'
        ),
        row=2, col=1
    )
    
    # Signal line
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data['signal_line'],
            mode='lines',
            line=dict(color='red'),
            name='Signal Line'
        ),
        row=2, col=1
    )
    
    # MACD histogram
    colors = ['green' if x > 0 else 'red' for x in data['histogram']]
    fig.add_trace(
        go.Bar(
            x=data.index,
            y=data['histogram'],
            marker_color=colors,
            name='Histogram'
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title=f'{symbol.upper()} MACD Crossover Strategy',
        xaxis_title='Date',
        yaxis_title='Price',
        height=800,
        width=1200
    )
    
    # Save to HTML
    fig.write_html(output_dir / f"{symbol}_price_signals.html")
    
    return fig

def plot_equity_curve(data, symbol, output_dir):
    """
    Plot equity curve
    
    Parameters:
    -----------
    data : pd.DataFrame
        DataFrame with backtest results
    symbol : str
        Trading symbol
    output_dir : Path
        Directory to save the plot
    """
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                         vertical_spacing=0.05, 
                         subplot_titles=(f'{symbol.upper()} Equity Curve', 'Drawdown'))
    
    # Equity curve
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data['capital'],
            mode='lines',
            line=dict(color='blue'),
            name='Equity'
        ),
        row=1, col=1
    )
    
    # Calculate drawdown
    data['peak'] = data['capital'].cummax()
    data['drawdown'] = (data['capital'] - data['peak']) / data['peak'] * 100
    
    # Drawdown
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data['drawdown'],
            mode='lines',
            line=dict(color='red'),
            name='Drawdown (%)'
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title=f'{symbol.upper()} Equity Curve and Drawdown',
        xaxis_title='Date',
        yaxis_title='Capital',
        height=800,
        width=1200
    )
    
    # Save to HTML
    fig.write_html(output_dir / f"{symbol}_equity_curve.html")
    
    return fig

# Performance metrics functions
def calculate_performance_metrics(backtest_data, trade_log, symbol, initial_capital):
    """
    Calculate performance metrics
    
    Parameters:
    -----------
    backtest_data : pd.DataFrame
        DataFrame with backtest results
    trade_log : pd.DataFrame
        DataFrame with trade logs
    symbol : str
        Trading symbol
    initial_capital : float
        Initial capital
        
    Returns:
    --------
    dict
        Dictionary with performance metrics
    """
    metrics = {}
    
    # Total return
    final_capital = backtest_data['capital'].iloc[-1]
    total_return = (final_capital - initial_capital) / initial_capital * 100
    metrics['total_return'] = total_return
    
    # Annualized return
    days = (backtest_data.index[-1] - backtest_data.index[0]).days
    annualized_return = ((1 + total_return / 100) ** (365 / days) - 1) * 100 if days > 0 else 0
    metrics['annualized_return'] = annualized_return
    
    # Maximum drawdown
    backtest_data['peak'] = backtest_data['capital'].cummax()
    backtest_data['drawdown'] = (backtest_data['capital'] - backtest_data['peak']) / backtest_data['peak'] * 100
    max_drawdown = backtest_data['drawdown'].min()
    metrics['max_drawdown'] = max_drawdown
    
    # Sharpe ratio (using daily returns, risk-free rate of 0)
    backtest_data['daily_return'] = backtest_data['capital'].pct_change()
    sharpe_ratio = np.sqrt(365) * backtest_data['daily_return'].mean() / backtest_data['daily_return'].std() if backtest_data['daily_return'].std() > 0 else 0
    metrics['sharpe_ratio'] = sharpe_ratio
    
    # Win rate
    if len(trade_log) > 0:
        winning_trades = trade_log[trade_log['pnl'] > 0]
        win_rate = len(winning_trades) / len(trade_log) * 100
        metrics['win_rate'] = win_rate
        
        # Loss rate
        losing_trades = trade_log[trade_log['pnl'] <= 0]
        loss_rate = len(losing_trades) / len(trade_log) * 100
        metrics['loss_rate'] = loss_rate
        
        # Average trade duration
        if 'entry_time' in trade_log.columns and 'exit_time' in trade_log.columns:
            trade_log['duration'] = (trade_log['exit_time'] - trade_log['entry_time']).dt.total_seconds() / 3600  # in hours
            avg_trade_duration = trade_log['duration'].mean()
            metrics['avg_trade_duration_hours'] = avg_trade_duration
        
        # Profit factor
        total_profit = winning_trades['pnl'].sum() if len(winning_trades) > 0 else 0
        total_loss = abs(losing_trades['pnl'].sum()) if len(losing_trades) > 0 else 0
        profit_factor = total_profit / total_loss if total_loss > 0 else float('inf')
        metrics['profit_factor'] = profit_factor
        
        # Expectancy
        avg_win = winning_trades['pnl'].mean() if len(winning_trades) > 0 else 0
        avg_loss = losing_trades['pnl'].mean() if len(losing_trades) > 0 else 0
        expectancy = (win_rate / 100 * avg_win) + (loss_rate / 100 * avg_loss)
        metrics['expectancy'] = expectancy
    else:
        metrics['win_rate'] = 0
        metrics['loss_rate'] = 0
        metrics['avg_trade_duration_hours'] = 0
        metrics['profit_factor'] = 0
        metrics['expectancy'] = 0
    
    return metrics

def save_performance_report(metrics, symbol, output_dir):
    """
    Save performance report
    
    Parameters:
    -----------
    metrics : dict
        Dictionary with performance metrics
    symbol : str
        Trading symbol
    output_dir : Path
        Directory to save the report
    """
    report = pd.DataFrame(metrics, index=[0])
    report.to_csv(output_dir / f"{symbol}_performance_report.csv", index=False)
    
    return report

def run_backtest(symbol, short_window=12, long_window=26, signal_window=9, 
                initial_capital=10000, position_size=1.0, 
                take_profit=None, stop_loss=None, trailing_stop=None):
    """
    Run backtest for a symbol
    
    Parameters:
    -----------
    symbol : str
        Trading symbol
    short_window : int
        Short EMA window
    long_window : int
        Long EMA window
    signal_window : int
        Signal EMA window
    initial_capital : float
        Initial capital
    position_size : float
        Position size as a percentage of capital (0-1)
    take_profit : float or None
        Take profit level as a percentage (e.g., 0.05 for 5%)
    stop_loss : float or None
        Stop loss level as a percentage (e.g., 0.03 for 3%)
    trailing_stop : float or None
        Trailing stop as a percentage (e.g., 0.02 for 2%)
        
    Returns:
    --------
    tuple
        Tuple with backtest results, trade log, and performance metrics
    """
    # Load data
    data_loader = DataLoader(symbol)
    data = data_loader.load_data()
    
    # Generate signals
    strategy = MACDStrategy(short_window, long_window, signal_window)
    signal_data = strategy.generate_signals(data)
    
    # Run backtest
    engine = BacktestEngine(initial_capital, position_size, take_profit, stop_loss, trailing_stop)
    backtest_data, trade_log = engine.run(signal_data)
    
    # Calculate performance metrics
    metrics = calculate_performance_metrics(backtest_data, trade_log, symbol, initial_capital)
    
    # Create visualizations
    plot_price_with_signals(backtest_data, symbol, OUTPUT_DIR)
    plot_equity_curve(backtest_data, symbol, OUTPUT_DIR)
    
    # Save results
    backtest_data.to_csv(OUTPUT_DIR / f"{symbol}_backtest_results.csv")
    if len(trade_log) > 0:
        trade_log.to_csv(OUTPUT_DIR / f"{symbol}_trade_log.csv", index=False)
    save_performance_report(metrics, symbol, OUTPUT_DIR)
    
    return backtest_data, trade_log, metrics

def main():
    """
    Main function to run the backtests
    """
    # Parameters
    short_window = 12
    long_window = 26
    signal_window = 9
    initial_capital = 10000
    position_size = 0.9  # 90% of capital
    take_profit = 0.1    # 10% take profit
    stop_loss = 0.05     # 5% stop loss
    trailing_stop = 0.03 # 3% trailing stop
    
    # Run backtest for BTC
    print("Running backtest for BTCUSD...")
    btc_results, btc_trades, btc_metrics = run_backtest(
        "btcusd", 
        short_window, 
        long_window, 
        signal_window,
        initial_capital,
        position_size,
        take_profit,
        stop_loss,
        trailing_stop
    )
    
    # Run backtest for ETH
    print("Running backtest for ETHUSD...")
    eth_results, eth_trades, eth_metrics = run_backtest(
        "ethusd", 
        short_window, 
        long_window, 
        signal_window,
        initial_capital,
        position_size,
        take_profit,
        stop_loss,
        trailing_stop
    )
    
    # Print summary
    print("\nBacktest Summary:")
    print("=================")
    print(f"BTC Total Return: {btc_metrics['total_return']:.2f}%")
    print(f"BTC Sharpe Ratio: {btc_metrics['sharpe_ratio']:.2f}")
    print(f"BTC Win Rate: {btc_metrics['win_rate']:.2f}%")
    print(f"BTC Max Drawdown: {btc_metrics['max_drawdown']:.2f}%")
    print("-----------------")
    print(f"ETH Total Return: {eth_metrics['total_return']:.2f}%")
    print(f"ETH Sharpe Ratio: {eth_metrics['sharpe_ratio']:.2f}")
    print(f"ETH Win Rate: {eth_metrics['win_rate']:.2f}%")
    print(f"ETH Max Drawdown: {eth_metrics['max_drawdown']:.2f}%")
    
    print("\nResults saved to the 'output' directory.")

if __name__ == "__main__":
    main()

Running backtest for BTCUSD...
Reindexed btcusd data to ensure regular 10-minute intervals
Running backtest for ETHUSD...
Reindexed ethusd data to ensure regular 10-minute intervals

Backtest Summary:
BTC Total Return: nan%
BTC Sharpe Ratio: 0.00
BTC Win Rate: 20.57%
BTC Max Drawdown: -100.00%
-----------------
ETH Total Return: nan%
ETH Sharpe Ratio: 0.00
ETH Win Rate: 17.13%
ETH Max Drawdown: -100.00%

Results saved to the 'output' directory.


In [3]:
# Comprehensive Crypto MACD Crossover Strategy Backtesting using vectorbt
# ========================================================================

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import vectorbt as vbt
from datetime import datetime, timezone, timedelta
import pytz
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure vectorbt settings safely (compatible with most versions)
try:
    vbt.settings.set_theme('light')
except:
    # Older versions might not have this setting
    pass

# %% [markdown]
# ## 1. Setup and Data Loading

# %%
# Set paths
DATA_DIR = Path("data")
OUTPUT_DIR = Path("output")
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# %%
class DataLoader:
    def __init__(self, symbol, timeframe="10m"):
        """
        Initialize the DataLoader with a symbol and timeframe
        """
        self.symbol = symbol.lower()
        self.timeframe = timeframe
        self.data = None
        
    def load_data(self):
        """
        Load data from CSV file and preprocess
        """
        file_path = DATA_DIR / f"{self.symbol}_{self.timeframe}.csv"
        
        # Check if file exists
        if not file_path.exists():
            raise FileNotFoundError(f"Data file not found: {file_path}")
        
        # Load data
        df = pd.read_csv(file_path)
        
        # Convert time columns to datetime
        df['time_utc'] = pd.to_datetime(df['time_utc'])
        df['time_est'] = pd.to_datetime(df['time_est'])
        
        # Set time_utc as index
        df.set_index('time_utc', inplace=True)
        
        # Check for missing intervals
        time_diff = df.index.to_series().diff().dt.total_seconds() / 60
        if not all(time_diff.dropna() == 10):
            print(f"Warning: Data for {self.symbol} has missing or irregular intervals")
            
        # Check for duplicates
        if df.index.duplicated().any():
            print(f"Warning: Duplicate timestamps found in {self.symbol} data")
            df = df[~df.index.duplicated()]
        
        # Check for null values
        if df[['o', 'h', 'l', 'c', 'v']].isnull().any().any():
            print(f"Warning: Null values found in {self.symbol} OHLCV data")
            # Fill missing values using forward fill
            df[['o', 'h', 'l', 'c', 'v']] = df[['o', 'h', 'l', 'c', 'v']].fillna(method='ffill')
        
        # Ensure regular 10-minute intervals
        start_time = df.index.min()
        end_time = df.index.max()
        full_range = pd.date_range(start=start_time, end=end_time, freq='10min')
        
        # Reindex and interpolate if needed
        if len(full_range) != len(df):
            df = df.reindex(full_range)
            df[['o', 'h', 'l', 'c', 'v']] = df[['o', 'h', 'l', 'c', 'v']].interpolate(method='linear')
            print(f"Reindexed {self.symbol} data to ensure regular 10-minute intervals")
        
        self.data = df
        return df
    
    def get_ohlcv_data(self):
        """
        Get OHLCV data compatible with all vectorbt versions
        """
        if self.data is None:
            self.load_data()
        
        return self.data[['o', 'h', 'l', 'c', 'v']]

# %% [markdown]
# ## 2. Load and Visualize Data for BTC and ETH

# %%
# Load data for BTC and ETH
btc_loader = DataLoader("btcusd")
eth_loader = DataLoader("ethusd")

# Get OHLCV data
btc_data = btc_loader.get_ohlcv_data()
eth_data = eth_loader.get_ohlcv_data()

# %% [markdown]
# ## 3. MACD Strategy Implementation with vectorbt

# %%
def run_macd_strategy(data, fast_window=12, slow_window=26, signal_window=9):
    """
    Generate MACD signals using vectorbt
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data
    fast_window : int
        Fast EMA window
    slow_window : int
        Slow EMA window
    signal_window : int
        Signal EMA window
    
    Returns:
    --------
    tuple
        (entries, exits, macd_indicator)
    """
    # Calculate MACD using vectorbt's built-in indicator
    macd_ind = vbt.indicators.MACD.run(
        data['c'], 
        fast_window=fast_window, 
        slow_window=slow_window, 
        signal_window=signal_window,
        short_name='macd'
    )
    
    # Generate signals using crossovers
    # Entries: MACD crosses above signal line
    entries = macd_ind.macd_above_signal()
    
    # Exits: MACD crosses below signal line
    exits = macd_ind.macd_below_signal()
    
    # Shift signals to execute on next candle's open
    entries_shifted = entries.vbt.signals.fshift(1)
    exits_shifted = exits.vbt.signals.fshift(1)
    
    return entries_shifted, exits_shifted, macd_ind

# %% [markdown]
# ## 4. Portfolio Backtesting with Risk Management

# %%
def run_backtest(data, entries, exits, 
                initial_capital=10000, position_size=1.0,
                take_profit=None, stop_loss=None, trailing_stop=None,
                fees=0.001, slippage=0.0005, freq='10T'):
    """
    Run backtest using vectorbt Portfolio
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data
    entries : pd.Series
        Entry signals (boolean)
    exits : pd.Series
        Exit signals (boolean)
    initial_capital : float
        Initial capital
    position_size : float
        Position size as a percentage of capital (0-1)
    take_profit : float or None
        Take profit level as a percentage (e.g., 0.05 for 5%)
    stop_loss : float or None
        Stop loss level as a percentage (e.g., 0.03 for 3%)
    trailing_stop : float or None
        Trailing stop as a percentage (e.g., 0.02 for 2%)
    fees : float
        Trading fee as a percentage (e.g., 0.001 for 0.1%)
    slippage : float
        Slippage as a percentage (e.g., 0.0005 for 0.05%)
    freq : str
        Data frequency
    
    Returns:
    --------
    vbt.Portfolio
        Portfolio object with backtest results
    """
    # Run backtest using vectorbt Portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=data['c'],
        entries=entries,
        exits=exits,
        init_cash=initial_capital,
        size=position_size,
        size_type='percent',
        fees=fees,
        slippage=slippage,
        open=data['o'],  # Use open prices for entries and exits
        high=data['h'],
        low=data['l'],
        # Risk management
        sl_stop=stop_loss,  # Stop loss as a percentage of entry price
        tp_stop=take_profit,  # Take profit as a percentage of entry price
        trailing_sl=trailing_stop,  # Trailing stop as a percentage of highest price since entry
        freq=freq
    )
    
    return portfolio

# %% [markdown]
# ## 5. Optimization Function for MACD Parameters

# %%
def optimize_macd_parameters(data, fast_windows, slow_windows, signal_windows,
                            initial_capital=10000, position_size=0.9,
                            take_profit=0.1, stop_loss=0.05, trailing_stop=0.03):
    """
    Optimize MACD parameters using vectorbt
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data
    fast_windows : list
        List of fast EMA windows to test
    slow_windows : list
        List of slow EMA windows to test
    signal_windows : list
        List of signal EMA windows to test
    Other parameters as in run_backtest
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with performance metrics for each parameter combination
    """
    # Storage for results
    results = []
    
    # Generate all parameter combinations
    for fast in fast_windows:
        for slow in slow_windows:
            if fast >= slow:  # Skip invalid combinations
                continue
            for signal in signal_windows:
                # Run MACD strategy with this parameter set
                entries, exits, macd_ind = run_macd_strategy(
                    data, fast_window=fast, slow_window=slow, signal_window=signal
                )
                
                # Run backtest
                portfolio = run_backtest(
                    data,
                    entries,
                    exits,
                    initial_capital=initial_capital,
                    position_size=position_size,
                    take_profit=take_profit,
                    stop_loss=stop_loss,
                    trailing_stop=trailing_stop
                )
                
                # Store results
                results.append({
                    'fast_window': fast,
                    'slow_window': slow,
                    'signal_window': signal,
                    'total_return': portfolio.total_return() * 100,
                    'sharpe_ratio': portfolio.sharpe_ratio(),
                    'max_drawdown': portfolio.max_drawdown() * 100,
                    'win_rate': portfolio.trades.win_rate() * 100 if portfolio.trades.count() > 0 else 0,
                    'profit_factor': portfolio.trades.profit_factor() if portfolio.trades.count() > 0 else 0,
                    'total_trades': portfolio.trades.count()
                })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by Sharpe ratio
    results_df.sort_values('sharpe_ratio', ascending=False, inplace=True)
    
    return results_df

# %% [markdown]
# ## 6. Complete Backtest Function

# %%
def full_macd_backtest(symbol, fast_window=12, slow_window=26, signal_window=9,
                      initial_capital=10000, position_size=0.9,
                      take_profit=0.1, stop_loss=0.05, trailing_stop=0.03,
                      optimize=False, save_results=True):
    """
    Complete MACD backtest workflow for a symbol
    
    Parameters:
    -----------
    symbol : str
        Trading symbol (e.g., 'btcusd', 'ethusd')
    fast_window, slow_window, signal_window : int
        MACD parameters
    initial_capital, position_size, take_profit, stop_loss, trailing_stop:
        Backtest parameters
    optimize : bool
        Whether to run parameter optimization
    save_results : bool
        Whether to save results to disk
    
    Returns:
    --------
    dict
        Dictionary with backtest results
    """
    # Load data
    loader = DataLoader(symbol)
    data = loader.get_ohlcv_data()
    
    print(f"Running MACD backtest for {symbol.upper()}...")
    
    # Optional parameter optimization
    opt_results = None
    if optimize:
        print(f"Optimizing parameters for {symbol.upper()}...")
        fast_windows = [8, 12, 16]
        slow_windows = [20, 26, 32]
        signal_windows = [7, 9, 11]
        
        opt_results = optimize_macd_parameters(
            data,
            fast_windows,
            slow_windows,
            signal_windows,
            initial_capital=initial_capital,
            position_size=position_size,
            take_profit=take_profit,
            stop_loss=stop_loss,
            trailing_stop=trailing_stop
        )
        
        # Get best parameters
        if len(opt_results) > 0:
            best_params = opt_results.iloc[0]
            print(f"Best parameters found: Fast={best_params['fast_window']}, "
                  f"Slow={best_params['slow_window']}, Signal={best_params['signal_window']}")
            
            # Use optimized parameters if available
            fast_window = int(best_params['fast_window'])
            slow_window = int(best_params['slow_window'])
            signal_window = int(best_params['signal_window'])
        
        if save_results and opt_results is not None:
            opt_results.to_csv(OUTPUT_DIR / f"{symbol}_optimization_results.csv", index=False)
    
    # Generate signals with final parameters
    entries, exits, macd_ind = run_macd_strategy(data, fast_window, slow_window, signal_window)
    
    # Run backtest
    portfolio = run_backtest(
        data, 
        entries, 
        exits,
        initial_capital=initial_capital,
        position_size=position_size,
        take_profit=take_profit,
        stop_loss=stop_loss,
        trailing_stop=trailing_stop
    )
    
    # Calculate performance metrics
    metrics = {
        'total_return': portfolio.total_return() * 100,
        'annual_return': portfolio.annual_return() * 100,
        'sharpe_ratio': portfolio.sharpe_ratio(),
        'sortino_ratio': portfolio.sortino_ratio(),
        'calmar_ratio': portfolio.calmar_ratio(),
        'max_drawdown': portfolio.max_drawdown() * 100,
        'win_rate': portfolio.trades.win_rate() * 100 if portfolio.trades.count() > 0 else 0,
        'profit_factor': portfolio.trades.profit_factor() if portfolio.trades.count() > 0 else 0,
        'avg_winning_trade': portfolio.trades.winning.pnl.mean() if portfolio.trades.winning.count() > 0 else 0,
        'avg_losing_trade': portfolio.trades.losing.pnl.mean() if portfolio.trades.losing.count() > 0 else 0,
        'total_trades': portfolio.trades.count()
    }
    
    # Save results if requested
    if save_results:
        # Save portfolio stats
        portfolio.stats().to_csv(OUTPUT_DIR / f"{symbol}_performance_report.csv")
        
        # Save trade log if there were trades
        if portfolio.trades.count() > 0:
            portfolio.trades.records_readable.to_csv(OUTPUT_DIR / f"{symbol}_trade_log.csv", index=False)
        
        # Save figures to HTML if available
        try:
            # Basic portfolio overview
            fig1 = portfolio.plot()
            fig1.write_html(OUTPUT_DIR / f"{symbol}_portfolio_overview.html")
            
            # Equity curve
            fig2 = portfolio.plot_cum_returns(benchmark_rets=data['c'].pct_change())
            fig2.write_html(OUTPUT_DIR / f"{symbol}_equity_curve.html")
            
            # Drawdowns
            fig3 = portfolio.plot_drawdowns()
            fig3.write_html(OUTPUT_DIR / f"{symbol}_drawdowns.html")
            
            # Save as PNG if HTML export fails
        except Exception as e:
            print(f"Warning: Could not save interactive HTML figures: {e}")
            # Try to save as static matplotlib figures
            try:
                plt.figure(figsize=(12, 8))
                portfolio.plot_cum_returns(benchmark_rets=data['c'].pct_change(), use_plt=True)
                plt.title(f"{symbol.upper()} Equity Curve")
                plt.savefig(OUTPUT_DIR / f"{symbol}_equity_curve.png")
                plt.close()
            except:
                print("Warning: Could not save static figures either")
    
    # Print summary
    print(f"\n{symbol.upper()} Backtest Summary:")
    print("=" * 30)
    print(f"Total Return: {metrics['total_return']:.2f}%")
    print(f"Annual Return: {metrics['annual_return']:.2f}%")
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
    print(f"Max Drawdown: {metrics['max_drawdown']:.2f}%")
    print(f"Win Rate: {metrics['win_rate']:.2f}%")
    print(f"Total Trades: {metrics['total_trades']}")
    
    return {
        'symbol': symbol,
        'portfolio': portfolio,
        'macd_indicator': macd_ind,
        'metrics': metrics,
        'optimization': opt_results
    }

# %% [markdown]
# ## 7. Main Function to Run Everything

# %%
def main():
    """
    Main function to run the complete backtest workflow
    """
    # Parameters
    initial_capital = 10000
    position_size = 0.9
    take_profit = 0.1
    stop_loss = 0.05
    trailing_stop = 0.03
    
    # Run backtest for BTC
    btc_results = full_macd_backtest(
        "btcusd",
        optimize=True,
        initial_capital=initial_capital,
        position_size=position_size,
        take_profit=take_profit,
        stop_loss=stop_loss,
        trailing_stop=trailing_stop
    )
    
    # Run backtest for ETH
    eth_results = full_macd_backtest(
        "ethusd",
        optimize=True,
        initial_capital=initial_capital,
        position_size=position_size,
        take_profit=take_profit,
        stop_loss=stop_loss,
        trailing_stop=trailing_stop
    )
    
    # Compare assets
    symbols = ['BTCUSD', 'ETHUSD']
    metrics_list = [btc_results['metrics'], eth_results['metrics']]
    
    comparison = pd.DataFrame({
        'Symbol': symbols,
        'Total Return (%)': [m['total_return'] for m in metrics_list],
        'Annual Return (%)': [m['annual_return'] for m in metrics_list],
        'Sharpe Ratio': [m['sharpe_ratio'] for m in metrics_list],
        'Max Drawdown (%)': [m['max_drawdown'] for m in metrics_list],
        'Win Rate (%)': [m['win_rate'] for m in metrics_list],
        'Calmar Ratio': [m['calmar_ratio'] for m in metrics_list],
        'Total Trades': [m['total_trades'] for m in metrics_list]
    })
    
    # Save comparison to CSV
    comparison.to_csv(OUTPUT_DIR / "asset_comparison.csv", index=False)
    
    print("\nAsset Comparison:")
    print("=" * 30)
    print(comparison)
    
    print("\nAll results saved to the 'output' directory.")
    
    return btc_results, eth_results

# Run the main function
if __name__ == "__main__":
    btc_results, eth_results = main()

Reindexed btcusd data to ensure regular 10-minute intervals
Reindexed ethusd data to ensure regular 10-minute intervals
Reindexed btcusd data to ensure regular 10-minute intervals
Running MACD backtest for BTCUSD...
Optimizing parameters for BTCUSD...


AttributeError: 'MACD' object has no attribute 'macd_above_signal'

In [7]:
# Crypto MACD Crossover Strategy Backtesting - Compatible Version
# ==============================================================

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import vectorbt as vbt
from datetime import datetime, timezone, timedelta
import pytz
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set paths
DATA_DIR = Path("data")
OUTPUT_DIR = Path("output")
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

class DataLoader:
    def __init__(self, symbol, timeframe="10m"):
        """
        Initialize the DataLoader with a symbol and timeframe
        """
        self.symbol = symbol.lower()
        self.timeframe = timeframe
        self.data = None
        
    def load_data(self):
        """
        Load data from CSV file and preprocess
        """
        file_path = DATA_DIR / f"{self.symbol}_{self.timeframe}.csv"
        
        # Check if file exists
        if not file_path.exists():
            raise FileNotFoundError(f"Data file not found: {file_path}")
        
        # Load data
        df = pd.read_csv(file_path)
        
        # Convert time columns to datetime
        df['time_utc'] = pd.to_datetime(df['time_utc'])
        df['time_est'] = pd.to_datetime(df['time_est'])
        
        # Set time_utc as index
        df.set_index('time_utc', inplace=True)
        
        # Check for missing intervals
        time_diff = df.index.to_series().diff().dt.total_seconds() / 60
        if not all(time_diff.dropna() == 10):
            print(f"Warning: Data for {self.symbol} has missing or irregular intervals")
            
        # Check for duplicates
        if df.index.duplicated().any():
            print(f"Warning: Duplicate timestamps found in {self.symbol} data")
            df = df[~df.index.duplicated()]
        
        # Check for null values
        if df[['o', 'h', 'l', 'c', 'v']].isnull().any().any():
            print(f"Warning: Null values found in {self.symbol} OHLCV data")
            # Fill missing values using forward fill
            df[['o', 'h', 'l', 'c', 'v']] = df[['o', 'h', 'l', 'c', 'v']].fillna(method='ffill')
        
        # Ensure regular 10-minute intervals
        start_time = df.index.min()
        end_time = df.index.max()
        full_range = pd.date_range(start=start_time, end=end_time, freq='10min')
        
        # Reindex and interpolate if needed
        if len(full_range) != len(df):
            df = df.reindex(full_range)
            df[['o', 'h', 'l', 'c', 'v']] = df[['o', 'h', 'l', 'c', 'v']].interpolate(method='linear')
            print(f"Reindexed {self.symbol} data to ensure regular 10-minute intervals")
        
        self.data = df
        return df
    
    def get_ohlcv_data(self):
        """
        Get OHLCV data compatible with all vectorbt versions
        """
        if self.data is None:
            self.load_data()
        
        return self.data[['o', 'h', 'l', 'c', 'v']]

def calculate_macd(price, fast_window=12, slow_window=26, signal_window=9):
    """
    Manual calculation of MACD components
    
    Parameters:
    -----------
    price : pd.Series
        Price series (typically close price)
    fast_window : int
        Fast EMA window
    slow_window : int
        Slow EMA window
    signal_window : int
        Signal EMA window
    
    Returns:
    --------
    tuple
        (macd, signal, histogram)
    """
    # Calculate EMAs
    fast_ema = price.ewm(span=fast_window, adjust=False).mean()
    slow_ema = price.ewm(span=slow_window, adjust=False).mean()
    
    # Calculate MACD line
    macd = fast_ema - slow_ema
    
    # Calculate signal line
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    
    # Calculate histogram
    histogram = macd - signal
    
    return macd, signal, histogram

def detect_crossovers(series1, series2):
    """
    Detect when series1 crosses above and below series2
    
    Parameters:
    -----------
    series1 : pd.Series
        First series (e.g., MACD line)
    series2 : pd.Series
        Second series (e.g., Signal line)
    
    Returns:
    --------
    tuple
        (crosses_above, crosses_below)
    """
    # Initialize with False
    crosses_above = pd.Series(False, index=series1.index)
    crosses_below = pd.Series(False, index=series1.index)
    
    # Previous state: 1 if series1 > series2, -1 if series1 < series2, 0 if equal
    prev_state = np.sign(series1 - series2).shift(1)
    curr_state = np.sign(series1 - series2)
    
    # Detect crossovers
    crosses_above[(prev_state < 0) & (curr_state > 0)] = True  # Was below, now above
    crosses_below[(prev_state > 0) & (curr_state < 0)] = True  # Was above, now below
    
    return crosses_above, crosses_below

def run_macd_strategy(data, fast_window=12, slow_window=26, signal_window=9):
    """
    Generate MACD signals with manual crossover detection
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data
    fast_window : int
        Fast EMA window
    slow_window : int
        Slow EMA window
    signal_window : int
        Signal EMA window
    
    Returns:
    --------
    tuple
        (entries, exits, macd_data)
    """
    # Calculate MACD components
    macd, signal, histogram = calculate_macd(
        data['c'], 
        fast_window=fast_window, 
        slow_window=slow_window, 
        signal_window=signal_window
    )
    
    # Detect crossovers
    crosses_above, crosses_below = detect_crossovers(macd, signal)
    
    # Create DataFrame with MACD data
    macd_data = pd.DataFrame({
        'macd': macd,
        'signal': signal,
        'histogram': histogram,
        'crosses_above': crosses_above,
        'crosses_below': crosses_below
    }, index=data.index)
    
    # Generate entries and exits
    entries = crosses_above
    exits = crosses_below
    
    # Shift signals to execute on next candle's open
    entries_shifted = entries.shift(1).fillna(False)
    exits_shifted = exits.shift(1).fillna(False)
    
    return entries_shifted, exits_shifted, macd_data

# def run_backtest(data, entries, exits, 
#                 initial_capital=10000, position_size=1.0,
#                 take_profit=None, stop_loss=None, trailing_stop=None,
#                 fees=0.001, slippage=0.0005, freq='10T'):
#     """
#     Run backtest using vectorbt Portfolio
    
#     Parameters:
#     -----------
#     data : pd.DataFrame
#         OHLCV data
#     entries : pd.Series
#         Entry signals (boolean)
#     exits : pd.Series
#         Exit signals (boolean)
#     initial_capital : float
#         Initial capital
#     position_size : float
#         Position size as a percentage of capital (0-1)
#     take_profit : float or None
#         Take profit level as a percentage (e.g., 0.05 for 5%)
#     stop_loss : float or None
#         Stop loss level as a percentage (e.g., 0.03 for 3%)
#     trailing_stop : float or None
#         Trailing stop as a percentage (e.g., 0.02 for 2%)
#     fees : float
#         Trading fee as a percentage (e.g., 0.001 for 0.1%)
#     slippage : float
#         Slippage as a percentage (e.g., 0.0005 for 0.05%)
#     freq : str
#         Data frequency
    
#     Returns:
#     --------
#     vbt.Portfolio
#         Portfolio object with backtest results
#     """
#     # Run backtest using vectorbt Portfolio
#     portfolio = vbt.Portfolio.from_signals(
#         close=data['c'],
#         entries=entries,
#         exits=exits,
#         init_cash=initial_capital,
#         size=position_size,
#         size_type='percent',
#         fees=fees,
#         slippage=slippage,
#         open=data['o'],  # Use open prices for entries and exits
#         high=data['h'],
#         low=data['l'],
#         # Risk management
#         sl_stop=stop_loss,  # Stop loss as a percentage of entry price
#         tp_stop=take_profit,  # Take profit as a percentage of entry price
#         trailing_sl=trailing_stop,  # Trailing stop as a percentage of highest price since entry
#         freq=freq
#     )
    
#     return portfolio

def run_backtest(data, entries, exits, 
                initial_capital=10000, position_size=1.0,
                take_profit=None, stop_loss=None, trailing_stop=None,
                fees=0.001, slippage=0.0005, freq='10T'):
    """
    Run backtest using vectorbt Portfolio
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data
    entries : pd.Series
        Entry signals (boolean)
    exits : pd.Series
        Exit signals (boolean)
    initial_capital : float
        Initial capital
    position_size : float
        Position size as a percentage of capital (0-1)
    take_profit : float or None
        Take profit level as a percentage (e.g., 0.05 for 5%)
    stop_loss : float or None
        Stop loss level as a percentage (e.g., 0.03 for 3%)
    trailing_stop : float or None
        Trailing stop as a percentage (e.g., 0.02 for 2%)
    fees : float
        Trading fee as a percentage (e.g., 0.001 for 0.1%)
    slippage : float
        Slippage as a percentage (e.g., 0.0005 for 0.05%)
    freq : str
        Data frequency
    
    Returns:
    --------
    vbt.Portfolio
        Portfolio object with backtest results
    """
    # Calculate sl_stop based on trailing_stop or stop_loss
    if trailing_stop is not None:
        sl_stop = -trailing_stop  # Negative for trailing stop below the highest price
        sl_trail = True
    elif stop_loss is not None:
        sl_stop = -stop_loss  # Negative for stop loss below entry price
        sl_trail = False
    else:
        sl_stop = None
        sl_trail = False
    
    # Run backtest using vectorbt Portfolio
    portfolio = vbt.Portfolio.from_signals(
        close=data['c'],
        entries=entries,
        exits=exits,
        init_cash=initial_capital,
        size=position_size,
        size_type='percent',
        fees=fees,
        slippage=slippage,
        open=data['o'],  # Use open prices for entries and exits
        high=data['h'],
        low=data['l'],
        # Risk management
        sl_stop=sl_stop,
        sl_trail=sl_trail,
        tp_stop=take_profit,
        freq=freq
    )
    
    return portfolio


def optimize_macd_parameters(data, fast_windows, slow_windows, signal_windows,
                            initial_capital=10000, position_size=0.9,
                            take_profit=0.1, stop_loss=0.05, trailing_stop=0.03):
    """
    Optimize MACD parameters by testing all combinations
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data
    fast_windows : list
        List of fast EMA windows to test
    slow_windows : list
        List of slow EMA windows to test
    signal_windows : list
        List of signal EMA windows to test
    Other parameters as in run_backtest
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with performance metrics for each parameter combination
    """
    # Storage for results
    results = []
    
    # Generate all parameter combinations
    for fast in fast_windows:
        for slow in slow_windows:
            if fast >= slow:  # Skip invalid combinations
                continue
            for signal in signal_windows:
                # Run MACD strategy with this parameter set
                entries, exits, macd_data = run_macd_strategy(
                    data, fast_window=fast, slow_window=slow, signal_window=signal
                )
                
                # Run backtest
                portfolio = run_backtest(
                    data,
                    entries,
                    exits,
                    initial_capital=initial_capital,
                    position_size=position_size,
                    take_profit=take_profit,
                    stop_loss=stop_loss,
                    trailing_stop=trailing_stop
                )
                
                # Store results
                results.append({
                    'fast_window': fast,
                    'slow_window': slow,
                    'signal_window': signal,
                    'total_return': portfolio.total_return() * 100,
                    'sharpe_ratio': portfolio.sharpe_ratio(),
                    'max_drawdown': portfolio.max_drawdown() * 100,
                    'win_rate': portfolio.trades.win_rate() * 100 if portfolio.trades.count() > 0 else 0,
                    'profit_factor': portfolio.trades.profit_factor() if portfolio.trades.count() > 0 else 0,
                    'total_trades': portfolio.trades.count()
                })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by Sharpe ratio
    results_df.sort_values('sharpe_ratio', ascending=False, inplace=True)
    
    return results_df

def plot_macd_backtest(data, macd_data, portfolio, symbol, save_dir=None):
    """
    Create basic matplotlib visualizations for the backtest
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data
    macd_data : pd.DataFrame
        MACD data with crossovers
    portfolio : vbt.Portfolio
        Portfolio object with backtest results
    symbol : str
        Trading symbol for plot titles
    save_dir : Path or None
        Directory to save plots, or None to display
        
    Returns:
    --------
    None
    """
    # Create figure with subplots
    fig, axs = plt.subplots(3, 1, figsize=(12, 16), gridspec_kw={'height_ratios': [2, 1, 1]})
    
    # Plot 1: Price with entry/exit points
    axs[0].plot(data.index, data['c'], label='Close Price', color='blue', alpha=0.5)
    
    # Plot trades from the portfolio
    if portfolio.trades.count() > 0:
        # Entries
        entry_points = portfolio.trades[['entry_idx', 'entry_price', 'size']]
        entry_points_long = entry_points[entry_points['size'] > 0]
        entry_points_short = entry_points[entry_points['size'] < 0]
        
        if len(entry_points_long) > 0:
            entry_dates_long = data.index[entry_points_long['entry_idx'].values]
            axs[0].scatter(entry_dates_long, entry_points_long['entry_price'], 
                         marker='^', color='green', s=100, label='Buy')
        
        if len(entry_points_short) > 0:
            entry_dates_short = data.index[entry_points_short['entry_idx'].values]
            axs[0].scatter(entry_dates_short, entry_points_short['entry_price'], 
                         marker='v', color='red', s=100, label='Sell')
        
        # Exits
        exit_points = portfolio.trades[['exit_idx', 'exit_price']]
        exit_dates = data.index[exit_points['exit_idx'].values]
        axs[0].scatter(exit_dates, exit_points['exit_price'], 
                     marker='X', color='black', s=80, label='Exit')
    
    axs[0].set_title(f'{symbol.upper()} Price Chart with Trades')
    axs[0].set_ylabel('Price')
    axs[0].legend()
    axs[0].grid(True)
    
    # Plot 2: MACD and Signal
    axs[1].plot(macd_data.index, macd_data['macd'], label='MACD', color='blue')
    axs[1].plot(macd_data.index, macd_data['signal'], label='Signal', color='red')
    axs[1].set_title('MACD and Signal Line')
    axs[1].set_ylabel('Value')
    axs[1].legend()
    axs[1].grid(True)
    
    # Plot 3: Histogram
    colors = ['green' if x > 0 else 'red' for x in macd_data['histogram']]
    axs[2].bar(macd_data.index, macd_data['histogram'], color=colors, label='Histogram')
    axs[2].set_title('MACD Histogram')
    axs[2].set_ylabel('Value')
    axs[2].set_xlabel('Date')
    axs[2].grid(True)
    
    plt.tight_layout()
    
    # Save or display
    if save_dir is not None:
        plt.savefig(save_dir / f"{symbol}_macd_analysis.png", dpi=300)
        plt.close()
    else:
        plt.show()
    
    # Equity curve plot
    plt.figure(figsize=(12, 6))
    portfolio.plot_cum_returns(use_plt=True)
    plt.title(f"{symbol.upper()} Equity Curve")
    plt.grid(True)
    
    # Save or display
    if save_dir is not None:
        plt.savefig(save_dir / f"{symbol}_equity_curve.png", dpi=300)
        plt.close()
    else:
        plt.show()
    
    # Drawdown plot
    plt.figure(figsize=(12, 6))
    
    # Calculate drawdown
    drawdown = (portfolio.cum_returns() - portfolio.cum_returns().cummax()) * 100
    
    plt.fill_between(drawdown.index, 0, drawdown, color='red', alpha=0.5)
    plt.plot(drawdown.index, drawdown, color='red', label='Drawdown %')
    plt.title(f"{symbol.upper()} Drawdown")
    plt.ylabel('Drawdown (%)')
    plt.xlabel('Date')
    plt.grid(True)
    plt.legend()
    
    # Save or display
    if save_dir is not None:
        plt.savefig(save_dir / f"{symbol}_drawdown.png", dpi=300)
        plt.close()
    else:
        plt.show()

def full_macd_backtest(symbol, fast_window=12, slow_window=26, signal_window=9,
                      initial_capital=10000, position_size=0.9,
                      take_profit=0.1, stop_loss=0.05, trailing_stop=0.03,
                      optimize=False, save_results=True):
    """
    Complete MACD backtest workflow for a symbol
    
    Parameters:
    -----------
    symbol : str
        Trading symbol (e.g., 'btcusd', 'ethusd')
    fast_window, slow_window, signal_window : int
        MACD parameters
    initial_capital, position_size, take_profit, stop_loss, trailing_stop:
        Backtest parameters
    optimize : bool
        Whether to run parameter optimization
    save_results : bool
        Whether to save results to disk
    
    Returns:
    --------
    dict
        Dictionary with backtest results
    """
    # Load data
    loader = DataLoader(symbol)
    data = loader.get_ohlcv_data()
    
    print(f"Running MACD backtest for {symbol.upper()}...")
    
    # Optional parameter optimization
    opt_results = None
    if optimize:
        print(f"Optimizing parameters for {symbol.upper()}...")
        fast_windows = [8, 12, 16]
        slow_windows = [20, 26, 32]
        signal_windows = [7, 9, 11]
        
        opt_results = optimize_macd_parameters(
            data,
            fast_windows,
            slow_windows,
            signal_windows,
            initial_capital=initial_capital,
            position_size=position_size,
            take_profit=take_profit,
            stop_loss=stop_loss,
            trailing_stop=trailing_stop
        )
        
        # Get best parameters
        if len(opt_results) > 0:
            best_params = opt_results.iloc[0]
            print(f"Best parameters found: Fast={best_params['fast_window']}, "
                  f"Slow={best_params['slow_window']}, Signal={best_params['signal_window']}")
            
            # Use optimized parameters if available
            fast_window = int(best_params['fast_window'])
            slow_window = int(best_params['slow_window'])
            signal_window = int(best_params['signal_window'])
        
        if save_results and opt_results is not None:
            opt_results.to_csv(OUTPUT_DIR / f"{symbol}_optimization_results.csv", index=False)
    
    # Generate signals with final parameters
    entries, exits, macd_data = run_macd_strategy(data, fast_window, slow_window, signal_window)
    
    # Run backtest
    portfolio = run_backtest(
        data, 
        entries, 
        exits,
        initial_capital=initial_capital,
        position_size=position_size,
        take_profit=take_profit,
        stop_loss=stop_loss,
        trailing_stop=trailing_stop
    )
    
    # Calculate performance metrics
    metrics = {
        'total_return': portfolio.total_return() * 100,
        'annual_return': portfolio.annual_return() * 100,
        'sharpe_ratio': portfolio.sharpe_ratio(),
        'sortino_ratio': portfolio.sortino_ratio(),
        'calmar_ratio': portfolio.calmar_ratio(),
        'max_drawdown': portfolio.max_drawdown() * 100,
        'win_rate': portfolio.trades.win_rate() * 100 if portfolio.trades.count() > 0 else 0,
        'profit_factor': portfolio.trades.profit_factor() if portfolio.trades.count() > 0 else 0,
        'avg_winning_trade': portfolio.trades.winning.pnl.mean() if portfolio.trades.winning.count() > 0 else 0,
        'avg_losing_trade': portfolio.trades.losing.pnl.mean() if portfolio.trades.losing.count() > 0 else 0,
        'total_trades': portfolio.trades.count()
    }
    
    # Save results if requested
    if save_results:
        # Save portfolio stats
        portfolio.stats().to_csv(OUTPUT_DIR / f"{symbol}_performance_report.csv")
        
        # Save trade log if there were trades
        if portfolio.trades.count() > 0:
            portfolio.trades.records_readable.to_csv(OUTPUT_DIR / f"{symbol}_trade_log.csv", index=False)
        
        # Generate and save static plots
        plot_macd_backtest(data, macd_data, portfolio, symbol, save_dir=OUTPUT_DIR)
    
    # Print summary
    print(f"\n{symbol.upper()} Backtest Summary:")
    print("=" * 30)
    print(f"Total Return: {metrics['total_return']:.2f}%")
    print(f"Annual Return: {metrics['annual_return']:.2f}%")
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.4f}")
    print(f"Max Drawdown: {metrics['max_drawdown']:.2f}%")
    print(f"Win Rate: {metrics['win_rate']:.2f}%")
    print(f"Total Trades: {metrics['total_trades']}")
    
    return {
        'symbol': symbol,
        'portfolio': portfolio,
        'macd_data': macd_data,
        'metrics': metrics,
        'optimization': opt_results
    }

def main():
    """
    Main function to run the complete backtest workflow
    """
    # Parameters
    initial_capital = 10000
    position_size = 0.9
    take_profit = 0.1
    stop_loss = 0.05
    trailing_stop = 0.03
    
    # Run backtest for BTC
    btc_results = full_macd_backtest(
        "btcusd",
        optimize=True,
        initial_capital=initial_capital,
        position_size=position_size,
        take_profit=take_profit,
        stop_loss=stop_loss,
        trailing_stop=trailing_stop
    )
    
    # Run backtest for ETH
    eth_results = full_macd_backtest(
        "ethusd",
        optimize=True,
        initial_capital=initial_capital,
        position_size=position_size,
        take_profit=take_profit,
        stop_loss=stop_loss,
        trailing_stop=trailing_stop
    )
    
    # Compare assets
    symbols = ['BTCUSD', 'ETHUSD']
    metrics_list = [btc_results['metrics'], eth_results['metrics']]
    
    comparison = pd.DataFrame({
        'Symbol': symbols,
        'Total Return (%)': [m['total_return'] for m in metrics_list],
        'Annual Return (%)': [m['annual_return'] for m in metrics_list],
        'Sharpe Ratio': [m['sharpe_ratio'] for m in metrics_list],
        'Max Drawdown (%)': [m['max_drawdown'] for m in metrics_list],
        'Win Rate (%)': [m['win_rate'] for m in metrics_list],
        'Calmar Ratio': [m['calmar_ratio'] for m in metrics_list],
        'Total Trades': [m['total_trades'] for m in metrics_list]
    })
    
    # Save comparison to CSV
    comparison.to_csv(OUTPUT_DIR / "asset_comparison.csv", index=False)
    
    print("\nAsset Comparison:")
    print("=" * 30)
    print(comparison)
    
    print("\nAll results saved to the 'output' directory.")
    
    return btc_results, eth_results

# Run the main function
if __name__ == "__main__":
    btc_results, eth_results = main()

Reindexed btcusd data to ensure regular 10-minute intervals
Running MACD backtest for BTCUSD...
Optimizing parameters for BTCUSD...


ValueError: Stop value must be 0 or greater

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import itertools
import warnings
from typing import Dict, List, Tuple, Union, Optional
warnings.filterwarnings('ignore')


class DataHandler:
    """
    Handles loading, preprocessing, and validation of cryptocurrency data.
    Ensures data integrity, proper time alignment, and gap handling.
    """
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.data = {}

    def load_data(self, symbol):
        """Load data from CSV file."""
        print(f"Loading data for {self.data_dir}...")
        file_path = os.path.join(self.data_dir, f"{symbol.lower()}_10m.csv")
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Data file for {symbol} not found at {file_path}")
        
        df = pd.read_csv(file_path)
        return df

    def preprocess_data(self, df):
        """
        Preprocess data to ensure integrity:
        - Convert timestamps to datetime
        - Check for missing intervals
        - Handle gaps through interpolation if needed
        - Verify no duplicate timestamps
        """
        # Convert timestamps to datetime
        df['time_utc'] = pd.to_datetime(df['time_utc'])
        df['time_est'] = pd.to_datetime(df['time_est'])
        
        # Set time_utc as index
        df = df.set_index('time_utc')
        
        # Check for duplicate timestamps
        if df.index.duplicated().any():
            print(f"Warning: Found {df.index.duplicated().sum()} duplicate timestamps. Removing duplicates.")
            df = df[~df.index.duplicated(keep='first')]
        
        # Check for missing intervals
        expected_interval = pd.Timedelta(minutes=10)
        time_diffs = df.index.to_series().diff()
        missing_intervals = time_diffs[time_diffs > expected_interval]
        
        if len(missing_intervals) > 0:
            print(f"Warning: Found {len(missing_intervals)} missing intervals.")
            
            # Create a complete time range
            full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='10min')
            
            # Reindex with the complete range
            df = df.reindex(full_range)
            
            # Interpolate missing values (optional, can be customized)
            numeric_cols = ['o', 'h', 'l', 'c', 'v']
            df[numeric_cols] = df[numeric_cols].interpolate(method='linear')
            
            # Forward fill non-numeric columns
            df['symbol'] = df['symbol'].ffill()
            df['time_est'] = df['time_est'].ffill()
            
        # Reset index for easier processing
        df = df.reset_index().rename(columns={'index': 'time_utc'})
        
        # Basic data validation
        assert not df.isnull().any().any(), "Data contains NaN values after preprocessing"
        
        return df
    
    def get_data(self, symbol, start_date=None, end_date=None):
        """
        Load and preprocess data for a specific symbol and time range.
        
        Args:
            symbol: Cryptocurrency symbol (e.g., 'btcusd')
            start_date: Optional start date filter
            end_date: Optional end date filter
            
        Returns:
            Preprocessed DataFrame
        """
        # Load data
        df = self.load_data(symbol)
        
        # Preprocess
        df = self.preprocess_data(df)
        
        # Filter by date range if specified
        if start_date:
            df = df[df['time_utc'] >= pd.to_datetime(start_date)]
        if end_date:
            df = df[df['time_utc'] <= pd.to_datetime(end_date)]
            
        # Store data in the instance
        self.data[symbol] = df
        
        return df


class MACDStrategy:
    """
    MACD (Moving Average Convergence Divergence) crossover strategy implementation.
    Generates signals based on MACD and signal line crossovers with customizable parameters.
    """
    def __init__(self, 
                fast_period: int = 12, 
                slow_period: int = 26, 
                signal_period: int = 9,
                signal_type: str = 'both'):
        """
        Initialize MACD strategy with customizable parameters.
        
        Args:
            fast_period: Fast EMA period
            slow_period: Slow EMA period
            signal_period: Signal line period
            signal_type: Type of signals to generate ('both', 'long', 'short')
        """
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.signal_period = signal_period
        self.signal_type = signal_type
        
    def calculate_macd(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate MACD and signal line based on close prices.
        
        Args:
            df: DataFrame with OHLCV data
            
        Returns:
            DataFrame with added MACD columns
        """
        close = df['c'].values
        
        # Calculate EMAs
        ema_fast = self._calculate_ema(close, self.fast_period)
        ema_slow = self._calculate_ema(close, self.slow_period)
        
        # Calculate MACD line
        macd_line = ema_fast - ema_slow
        
        # Calculate signal line
        signal_line = self._calculate_ema(macd_line, self.signal_period)
        
        # Calculate histogram
        histogram = macd_line - signal_line
        
        # Add to dataframe
        df = df.copy()
        df['macd'] = macd_line
        df['macd_signal'] = signal_line
        df['macd_hist'] = histogram
        
        return df
    
    def _calculate_ema(self, values: np.ndarray, period: int) -> np.ndarray:
        """
        Calculate Exponential Moving Average.
        
        Args:
            values: Price series
            period: EMA period
            
        Returns:
            EMA values as numpy array
        """
        if len(values) < period:
            raise ValueError(f"Not enough data points for EMA calculation. Need at least {period}.")
        
        # Initialize with SMA for first period points
        ema = np.zeros_like(values)
        ema[:period] = np.nan
        ema[period-1] = np.mean(values[:period])
        
        # Calculate multiplier
        multiplier = 2 / (period + 1)
        
        # Calculate EMA
        for i in range(period, len(values)):
            ema[i] = (values[i] - ema[i-1]) * multiplier + ema[i-1]
            
        return ema
    
    def generate_signals(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Generate trading signals based on MACD crossovers.
        
        Args:
            df: DataFrame with OHLCV data
            
        Returns:
            DataFrame with added signal columns
        """
        # Calculate MACD if not already present
        if 'macd' not in df.columns:
            df = self.calculate_macd(df)
        
        # Initialize signal column
        df['signal'] = 0  # 0: no signal, 1: buy, -1: sell
        
        # Calculate crossovers (with shift to ensure signal is based on previous candle)
        df['macd_cross_above'] = (df['macd'] > df['macd_signal']) & (df['macd'].shift(1) <= df['macd_signal'].shift(1))
        df['macd_cross_below'] = (df['macd'] < df['macd_signal']) & (df['macd'].shift(1) >= df['macd_signal'].shift(1))
        
        # Generate signals based on signal_type
        if self.signal_type in ['both', 'long']:
            df.loc[df['macd_cross_above'], 'signal'] = 1
            
        if self.signal_type in ['both', 'short']:
            df.loc[df['macd_cross_below'], 'signal'] = -1
        
        return df
    
    def optimize_parameters(self, df: pd.DataFrame, metric: str = 'profit_factor') -> Dict:
        """
        Find optimal MACD parameters by grid search.
        
        Args:
            df: DataFrame with OHLCV data
            metric: Performance metric to optimize ('profit_factor', 'sharpe_ratio', etc.)
            
        Returns:
            Dictionary with optimal parameters and performance metrics
        """
        # Define parameter ranges
        fast_range = range(8, 21, 2)  # 8-20 by step of 2
        slow_range = range(20, 31, 2)  # 20-30 by step of 2
        signal_range = range(5, 14, 2)  # 5-13 by step of 2
        
        best_metric = -np.inf if metric == 'profit_factor' or metric == 'sharpe_ratio' else np.inf
        best_params = {}
        results = []
        
        # Perform grid search
        for fast, slow, signal in itertools.product(fast_range, slow_range, signal_range):
            if fast >= slow:  # Skip invalid combinations
                continue
                
            # Set parameters
            self.fast_period = fast
            self.slow_period = slow
            self.signal_period = signal
            
            # Generate signals
            df_signals = self.generate_signals(df)
            
            # Simulate basic trading
            backtester = Backtester(position_size=1.0)
            performance = backtester.run(df_signals)
            
            # Evaluate performance
            if metric == 'profit_factor' and performance['profit_factor'] > best_metric:
                best_metric = performance['profit_factor']
                best_params = {'fast': fast, 'slow': slow, 'signal': signal}
            elif metric == 'sharpe_ratio' and performance['sharpe_ratio'] > best_metric:
                best_metric = performance['sharpe_ratio']
                best_params = {'fast': fast, 'slow': slow, 'signal': signal}
            # Add more metrics as needed
            
            # Store result
            result = {
                'fast_period': fast,
                'slow_period': slow,
                'signal_period': signal,
                **performance
            }
            results.append(result)
        
        # Reset to best parameters
        if best_params:
            self.fast_period = best_params['fast']
            self.slow_period = best_params['slow']
            self.signal_period = best_params['signal']
        
        # Create results dataframe
        results_df = pd.DataFrame(results)
        
        return {
            'best_parameters': best_params,
            'best_metric_value': best_metric,
            'results': results_df
        }


class RiskManager:
    """
    Manages trade risk with features like take profit, stop loss, and trailing stop loss.
    Dynamically recalibrates exit conditions on each candle.
    """
    def __init__(self, 
                take_profit_pct: float = 0.05, 
                stop_loss_pct: float = 0.03,
                trailing_stop_pct: Optional[float] = None,
                time_stop: Optional[int] = None):
        """
        Initialize risk manager with risk parameters.
        
        Args:
            take_profit_pct: Take profit percentage
            stop_loss_pct: Stop loss percentage
            trailing_stop_pct: Trailing stop percentage (None to disable)
            time_stop: Maximum number of candles to hold a position (None to disable)
        """
        self.take_profit_pct = take_profit_pct
        self.stop_loss_pct = stop_loss_pct
        self.trailing_stop_pct = trailing_stop_pct
        self.time_stop = time_stop
        
        # Trade state
        self.entry_price = None
        self.position_type = None  # 'long' or 'short'
        self.highest_price = None  # For trailing stop on long
        self.lowest_price = None   # For trailing stop on short
        self.entry_time = None
        self.candles_in_position = 0
        
    def enter_position(self, price: float, position_type: str, timestamp=None):
        """Record position entry details for risk management"""
        self.entry_price = price
        self.position_type = position_type
        self.highest_price = price if position_type == 'long' else None
        self.lowest_price = price if position_type == 'short' else None
        self.entry_time = timestamp
        self.candles_in_position = 0
        
    def reset(self):
        """Reset risk manager state after exit"""
        self.entry_price = None
        self.position_type = None
        self.highest_price = None
        self.lowest_price = None
        self.entry_time = None
        self.candles_in_position = 0
        
    def update(self, current_price: float):
        """Update tracking variables for trailing stop loss"""
        if self.position_type == 'long' and current_price > self.highest_price:
            self.highest_price = current_price
        elif self.position_type == 'short' and (self.lowest_price is None or current_price < self.lowest_price):
            self.lowest_price = current_price
            
        self.candles_in_position += 1
        
    def check_exit_conditions(self, high: float, low: float, close: float) -> bool:
        """
        Check if any exit conditions are met.
        
        Args:
            high: High price of current candle
            low: Low price of current candle
            close: Close price of current candle
            
        Returns:
            Boolean indicating whether to exit position
        """
        if self.entry_price is None or self.position_type is None:
            return False
            
        # Update tracking prices
        if self.position_type == 'long':
            self.highest_price = max(self.highest_price, high)
        else:  # short
            self.lowest_price = min(self.lowest_price, low)
            
        # Check take profit
        if self.position_type == 'long' and high >= self.entry_price * (1 + self.take_profit_pct):
            return True
        elif self.position_type == 'short' and low <= self.entry_price * (1 - self.take_profit_pct):
            return True
            
        # Check stop loss
        if self.position_type == 'long' and low <= self.entry_price * (1 - self.stop_loss_pct):
            return True
        elif self.position_type == 'short' and high >= self.entry_price * (1 + self.stop_loss_pct):
            return True
            
        # Check trailing stop loss
        if self.trailing_stop_pct and self.position_type == 'long':
            trailing_stop_level = self.highest_price * (1 - self.trailing_stop_pct)
            if low <= trailing_stop_level:
                return True
                
        elif self.trailing_stop_pct and self.position_type == 'short':
            trailing_stop_level = self.lowest_price * (1 + self.trailing_stop_pct)
            if high >= trailing_stop_level:
                return True
                
        # Check time stop
        if self.time_stop and self.candles_in_position >= self.time_stop:
            return True
            
        return False


class Backtester:
    """
    Backtesting engine for trading strategies.
    Simulates trading with proper signal execution, risk management, and performance tracking.
    """
    def __init__(self, 
                initial_capital: float = 10000.0,
                position_size: float = 1.0,
                commission_pct: float = 0.001,
                take_profit_pct: float = 0.05,
                stop_loss_pct: float = 0.03,
                trailing_stop_pct: Optional[float] = 0.02,
                time_stop: Optional[int] = None):
        """
        Initialize backtester with trading parameters.
        
        Args:
            initial_capital: Starting capital
            position_size: Position size as fraction of capital (0-1)
            commission_pct: Commission per trade (%)
            take_profit_pct: Take profit level (%)
            stop_loss_pct: Stop loss level (%)
            trailing_stop_pct: Trailing stop level (%)
            time_stop: Max position duration in candles
        """
        self.initial_capital = initial_capital
        self.position_size = position_size
        self.commission_pct = commission_pct
        
        # Initialize risk manager
        self.risk_manager = RiskManager(
            take_profit_pct=take_profit_pct,
            stop_loss_pct=stop_loss_pct,
            trailing_stop_pct=trailing_stop_pct,
            time_stop=time_stop
        )
        
        # Trading state
        self.position = 0  # 0: flat, 1: long, -1: short
        self.entry_price = None
        self.entry_time = None
        self.exit_price = None
        self.exit_time = None
        self.exit_reason = None
        
        # Performance tracking
        self.capital = initial_capital
        self.equity_curve = []
        self.trades = []
        
    def run(self, df: pd.DataFrame) -> Dict:
        """
        Run backtest on dataframe with signals.
        
        Args:
            df: DataFrame with OHLCV data and signals
            
        Returns:
            Dictionary with performance metrics
        """
        # Reset state
        self.capital = self.initial_capital
        self.position = 0
        self.entry_price = None
        self.entry_time = None
        self.exit_price = None
        self.exit_time = None
        self.exit_reason = None
        self.equity_curve = []
        self.trades = []
        self.risk_manager.reset()
        
        # Ensure we have signals column
        if 'signal' not in df.columns:
            raise ValueError("DataFrame must contain 'signal' column with trading signals")
            
        # Add equity column to track performance
        df = df.copy()
        df['equity'] = self.initial_capital
        
        # Main backtest loop
        for i in range(1, len(df)):
            prev_row = df.iloc[i-1]
            curr_row = df.iloc[i]
            
            # Get signal from previous candle
            signal = prev_row['signal']
            
            # Current candle prices
            curr_open = curr_row['o']
            curr_high = curr_row['h']
            curr_low = curr_row['l']
            curr_close = curr_row['c']
            timestamp = curr_row['time_utc'] if 'time_utc' in curr_row.index else i
            
            # Check for position exit based on risk management
            if self.position != 0:
                # Update risk manager state
                self.risk_manager.update(curr_close)
                
                # Check exit conditions
                if self.risk_manager.check_exit_conditions(curr_high, curr_low, curr_close):
                    # Determine exit price based on exit reason
                    if self.position == 1:  # Long position
                        if curr_low <= self.entry_price * (1 - self.risk_manager.stop_loss_pct):
                            exit_price = self.entry_price * (1 - self.risk_manager.stop_loss_pct)
                            exit_reason = "stop_loss"
                        elif self.risk_manager.trailing_stop_pct and curr_low <= self.risk_manager.highest_price * (1 - self.risk_manager.trailing_stop_pct):
                            exit_price = self.risk_manager.highest_price * (1 - self.risk_manager.trailing_stop_pct)
                            exit_reason = "trailing_stop"
                        elif curr_high >= self.entry_price * (1 + self.risk_manager.take_profit_pct):
                            exit_price = self.entry_price * (1 + self.risk_manager.take_profit_pct)
                            exit_reason = "take_profit"
                        elif self.risk_manager.time_stop and self.risk_manager.candles_in_position >= self.risk_manager.time_stop:
                            exit_price = curr_open
                            exit_reason = "time_stop"
                        else:
                            exit_price = curr_open
                            exit_reason = "signal_exit"
                    else:  # Short position
                        if curr_high >= self.entry_price * (1 + self.risk_manager.stop_loss_pct):
                            exit_price = self.entry_price * (1 + self.risk_manager.stop_loss_pct)
                            exit_reason = "stop_loss"
                        elif self.risk_manager.trailing_stop_pct and curr_high >= self.risk_manager.lowest_price * (1 + self.risk_manager.trailing_stop_pct):
                            exit_price = self.risk_manager.lowest_price * (1 + self.risk_manager.trailing_stop_pct)
                            exit_reason = "trailing_stop"
                        elif curr_low <= self.entry_price * (1 - self.risk_manager.take_profit_pct):
                            exit_price = self.entry_price * (1 - self.risk_manager.take_profit_pct)
                            exit_reason = "take_profit"
                        elif self.risk_manager.time_stop and self.risk_manager.candles_in_position >= self.risk_manager.time_stop:
                            exit_price = curr_open
                            exit_reason = "time_stop"
                        else:
                            exit_price = curr_open
                            exit_reason = "signal_exit"
                    
                    # Calculate P&L
                    if self.position == 1:  # Long position
                        profit = (exit_price / self.entry_price - 1) * self.position_size * self.capital
                    else:  # Short position
                        profit = (self.entry_price / exit_price - 1) * self.position_size * self.capital
                        
                    # Subtract commission
                    commission = exit_price * self.position_size * self.capital * self.commission_pct
                    profit -= commission
                    
                    # Update capital
                    self.capital += profit
                    
                    # Record trade
                    trade = {
                        'entry_time': self.entry_time,
                        'entry_price': self.entry_price,
                        'exit_time': timestamp,
                        'exit_price': exit_price,
                        'position': 'long' if self.position == 1 else 'short',
                        'profit': profit,
                        'profit_pct': profit / (self.position_size * self.capital) * 100,
                        'exit_reason': exit_reason,
                        'duration': self.risk_manager.candles_in_position
                    }
                    self.trades.append(trade)
                    
                    # Reset position
                    self.position = 0
                    self.entry_price = None
                    self.entry_time = None
                    self.risk_manager.reset()
            
            # Check for new position entry (signals executed on next candle's open)
            if self.position == 0 and signal != 0:
                self.position = signal
                self.entry_price = curr_open
                self.entry_time = timestamp
                
                # Initialize risk manager
                position_type = 'long' if self.position == 1 else 'short'
                self.risk_manager.enter_position(self.entry_price, position_type, timestamp)
                
                # Deduct commission
                commission = curr_open * self.position_size * self.capital * self.commission_pct
                self.capital -= commission
            
            # Update equity curve
            df.loc[df.index[i], 'equity'] = self.capital
            self.equity_curve.append(self.capital)
            
        # Calculate performance metrics
        performance = self._calculate_performance()
        
        # Return results
        return {
            'df': df,
            'trades': pd.DataFrame(self.trades) if self.trades else pd.DataFrame(),
            'equity_curve': self.equity_curve,
            **performance
        }
    
    def _calculate_performance(self) -> Dict:
        """
        Calculate trading performance metrics.
        
        Returns:
            Dictionary with performance metrics
        """
        if not self.trades:
            return {
                'total_return': 0,
                'total_return_pct': 0,
                'annual_return_pct': 0,
                'max_drawdown_pct': 0,
                'win_rate': 0,
                'profit_factor': 0,
                'sharpe_ratio': 0,
                'total_trades': 0,
                'winning_trades': 0,
                'losing_trades': 0
            }
            
        # Convert trades to DataFrame for easier analysis
        trades_df = pd.DataFrame(self.trades)
        
        # Total return
        total_return = self.capital - self.initial_capital
        total_return_pct = (total_return / self.initial_capital) * 100
        
        # Annual return
        if len(self.trades) > 1:
            start_date = self.trades[0]['entry_time']
            end_date = self.trades[-1]['exit_time']
            if isinstance(start_date, (datetime, pd.Timestamp)) and isinstance(end_date, (datetime, pd.Timestamp)):
                years = (end_date - start_date).days / 365.25
                if years > 0:
                    annual_return_pct = ((1 + total_return_pct/100) ** (1/years) - 1) * 100
                else:
                    annual_return_pct = total_return_pct
            else:
                annual_return_pct = total_return_pct
        else:
            annual_return_pct = total_return_pct
        
        # Maximum drawdown
        equity_series = pd.Series(self.equity_curve)
        max_equity = equity_series.cummax()
        drawdown = (equity_series - max_equity) / max_equity * 100
        max_drawdown_pct = abs(drawdown.min())
        
        # Win rate
        winning_trades = len(trades_df[trades_df['profit'] > 0])
        losing_trades = len(trades_df[trades_df['profit'] <= 0])
        total_trades = len(trades_df)
        win_rate = winning_trades / total_trades if total_trades > 0 else 0
        
        # Profit factor
        gross_profits = trades_df[trades_df['profit'] > 0]['profit'].sum() if winning_trades > 0 else 0
        gross_losses = abs(trades_df[trades_df['profit'] <= 0]['profit'].sum()) if losing_trades > 0 else 0
        profit_factor = gross_profits / gross_losses if gross_losses > 0 else float('inf')
        
        # Sharpe ratio
        if len(trades_df) > 1:
            returns = trades_df['profit_pct'].values
            sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252) if np.std(returns) > 0 else 0
        else:
            sharpe_ratio = 0
            
        # Return metrics
        return {
            'total_return': total_return,
            'total_return_pct': total_return_pct,
            'annual_return_pct': annual_return_pct,
            'max_drawdown_pct': max_drawdown_pct,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'sharpe_ratio': sharpe_ratio,
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades
        }


class Visualizer:
    """
    Visualization utilities for crypto trading strategies.
    Creates interactive charts and performance visualizations.
    """
    @staticmethod
    def plot_equity_curve(equity_curve, title="Equity Curve"):
        """Plot equity curve using Plotly"""
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            y=equity_curve,
            mode='lines',
            name='Equity',
            line=dict(color='blue', width=2)
        ))
        
        fig.update_layout(
            title=title,
            xaxis_title="Time",
            yaxis_title="Equity",
            template="plotly_white"
        )
        
        return fig
    
    @staticmethod
    def plot_drawdown(equity_curve, title="Drawdown Analysis"):
        """Plot drawdown chart"""
        equity_series = pd.Series(equity_curve)
        max_equity = equity_series.cummax()
        drawdown = (equity_series - max_equity) / max_equity * 100
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            y=drawdown,
            mode='lines',
            name='Drawdown',
            line=dict(color='red', width=2),
            fill='tozeroy'
        ))
        
        fig.update_layout(
            title=title,
            xaxis_title="Time",
            yaxis_title="Drawdown (%)",
            template="plotly_white"
        )
        
        return fig
    
    @staticmethod
    def plot_trades(df, trades, title="Trade Analysis"):
        """Plot price chart with trade entries and exits"""
        # Create figure with secondary y-axis
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                           vertical_spacing=0.02, 
                           row_heights=[0.7, 0.3],
                           subplot_titles=["Price Chart with Trades", "MACD"])
        
        # Add candlestick chart
        fig.add_trace(go.Candlestick(
            x=df['time_utc'] if 'time_utc' in df.columns else df.index,
            open=df['o'],
            high=df['h'],
            low=df['l'],
            close=df['c'],
            name="OHLC"
        ), row=1, col=1)
        
        # Add entry and exit points if trades exists
        if len(trades) > 0:
            for _, trade in trades.iterrows():
                # Long entry
                if trade['position'] == 'long':
                    fig.add_trace(go.Scatter(
                        x=[trade['entry_time']],
                        y=[trade['entry_price']],
                        mode='markers',
                        marker=dict(color='green', size=10, symbol='triangle-up'),
                        name="Long Entry"
                    ), row=1, col=1)
                    
                    # Exit
                    fig.add_trace(go.Scatter(
                        x=[trade['exit_time']],
                        y=[trade['exit_price']],
                        mode='markers',
                        marker=dict(color='black', size=10, symbol='circle'),
                        name=f"Exit ({trade['exit_reason']})"
                    ), row=1, col=1)
                    
                # Short entry
                elif trade['position'] == 'short':
                    fig.add_trace(go.Scatter(
                        x=[trade['entry_time']],
                        y=[trade['entry_price']],
                        mode='markers',
                        marker=dict(color='red', size=10, symbol='triangle-down'),
                        name="Short Entry"
                    ), row=1, col=1)
                    
                    # Exit
                    fig.add_trace(go.Scatter(
                        x=[trade['exit_time']],
                        y=[trade['exit_price']],
                        mode='markers',
                        marker=dict(color='black', size=10, symbol='circle'),
                        name=f"Exit ({trade['exit_reason']})"
                    ), row=1, col=1)
        
        # Add MACD
        if all(col in df.columns for col in ['macd', 'macd_signal']):
            # MACD Line
            fig.add_trace(go.Scatter(
                x=df['time_utc'] if 'time_utc' in df.columns else df.index,
                y=df['macd'],
                mode='lines',
                name='MACD',
                line=dict(color='blue', width=2)
            ), row=2, col=1)
            
            # Signal Line
            fig.add_trace(go.Scatter(
                x=df['time_utc'] if 'time_utc' in df.columns else df.index,
                y=df['macd_signal'],
                mode='lines',
                name='Signal',
                line=dict(color='red', width=2)
            ), row=2, col=1)
            
            # Histogram
            colors = ['green' if val >= 0 else 'red' for val in df['macd_hist']]
            fig.add_trace(go.Bar(
                x=df['time_utc'] if 'time_utc' in df.columns else df.index,
                y=df['macd_hist'],
                name='Histogram',
                marker_color=colors
            ), row=2, col=1)
        
        # Update layout
        fig.update_layout(
            title=title,
            xaxis_title="Time",
            template="plotly_white",
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            height=800
        )
        
        # X-axis settings
        fig.update_xaxes(
            rangeslider_visible=False,
            row=1, col=1
        )
        
        # Y-axis settings
        fig.update_yaxes(title_text="Price", row=1, col=1)
        fig.update_yaxes(title_text="MACD", row=2, col=1)
        
        return fig
    
    @staticmethod
    def plot_performance_summary(results, title="Performance Summary"):
        """Plot performance metrics in a summary chart"""
        # Extract key metrics
        metrics = {
            'Total Return (%)': results['total_return_pct'],
            'Annual Return (%)': results['annual_return_pct'],
            'Max Drawdown (%)': results['max_drawdown_pct'],
            'Win Rate': results['win_rate'] * 100,
            'Profit Factor': results['profit_factor'],
            'Sharpe Ratio': results['sharpe_ratio']
        }
        
        # Create figure
        fig = go.Figure()
        
        # Add bar chart
        fig.add_trace(go.Bar(
            x=list(metrics.keys()),
            y=list(metrics.values()),
            text=[f"{v:.2f}" for v in metrics.values()],
            textposition='auto',
            marker_color=['green' if v > 0 else 'red' for v in metrics.values()]
        ))
        
        # Update layout
        fig.update_layout(
            title=title,
            xaxis_title="Metric",
            yaxis_title="Value",
            template="plotly_white"
        )
        
        return fig
    
    @staticmethod
    def plot_trade_distribution(trades, title="Trade Distribution"):
        """Plot trade profit/loss distribution"""
        if len(trades) == 0:
            return None
            
        fig = make_subplots(rows=1, cols=2, 
                           subplot_titles=["Profit/Loss Distribution", "Exit Reasons"],
                           specs=[[{"type": "histogram"}, {"type": "pie"}]])
        
        # Profit/Loss histogram
        fig.add_trace(go.Histogram(
            x=trades['profit_pct'],
            name='P&L',
            marker_color='blue',
            opacity=0.75
        ), row=1, col=1)
        
        # Exit reasons pie chart
        exit_reasons = trades['exit_reason'].value_counts()
        fig.add_trace(go.Pie(
            labels=exit_reasons.index,
            values=exit_reasons.values,
            name='Exit Reasons'
        ), row=1, col=2)
        
        # Update layout
        fig.update_layout(
            title=title,
            template="plotly_white"
        )
        
        return fig


class CryptoTradingSystem:
    """
    Main trading system class integrating data handling, strategy, backtesting, and visualization.
    """
    def __init__(self, data_dir=os.getcwd()):
        """
        Initialize the trading system.
        
        Args:
            data_dir: Directory containing cryptocurrency data
        """
        self.data_dir = data_dir
        self.data_handler = DataHandler(data_dir)
        self.strategy = None
        self.backtester = None
        self.results = {}
        
    def set_strategy(self, strategy_type="macd", **params):
        """
        Set and configure trading strategy.
        
        Args:
            strategy_type: Type of strategy ('macd', etc.)
            **params: Strategy parameters
        """
        if strategy_type.lower() == "macd":
            self.strategy = MACDStrategy(**params)
        else:
            raise ValueError(f"Unsupported strategy type: {strategy_type}")
            
    def set_backtester(self, **params):
        """
        Configure backtester with parameters.
        
        Args:
            **params: Backtester parameters
        """
        self.backtester = Backtester(**params)
        
    def run_backtest(self, symbol, start_date=None, end_date=None):
        """
        Run backtest on specified symbol and date range.
        
        Args:
            symbol: Cryptocurrency symbol (e.g., 'btcusd')
            start_date: Start date for backtest
            end_date: End date for backtest
            
        Returns:
            Dictionary with backtest results
        """
        # Ensure strategy and backtester are set
        if self.strategy is None:
            self.set_strategy("macd")
        if self.backtester is None:
            self.set_backtester()
            
        # Load and preprocess data
        df = self.data_handler.get_data(symbol, start_date, end_date)
        
        # Generate trading signals
        df = self.strategy.generate_signals(df)
        
        # Run backtest
        results = self.backtester.run(df)
        
        # Store results
        self.results[symbol] = results
        
        return results
    
    def optimize_strategy(self, symbol, start_date=None, end_date=None, metric='profit_factor'):
        """
        Optimize strategy parameters for a symbol.
        
        Args:
            symbol: Cryptocurrency symbol
            start_date: Start date for optimization
            end_date: End date for optimization
            metric: Performance metric to optimize
            
        Returns:
            Dictionary with optimization results
        """
        # Ensure strategy is set
        if self.strategy is None:
            self.set_strategy("macd")
            
        # Load and preprocess data
        df = self.data_handler.get_data(symbol, start_date, end_date)
        
        # Optimize parameters
        optimization_results = self.strategy.optimize_parameters(df, metric)
        
        return optimization_results
    
    def visualize_results(self, symbol):
        """
        Create visualizations for backtest results.
        
        Args:
            symbol: Cryptocurrency symbol
            
        Returns:
            Dictionary with visualization figures
        """
        if symbol not in self.results:
            raise ValueError(f"No results found for {symbol}. Run backtest first.")
            
        results = self.results[symbol]
        
        # Create visualizations
        visualizations = {
            'equity_curve': Visualizer.plot_equity_curve(results['equity_curve'], f"{symbol} Equity Curve"),
            'drawdown': Visualizer.plot_drawdown(results['equity_curve'], f"{symbol} Drawdown Analysis"),
            'trades': Visualizer.plot_trades(results['df'], results['trades'], f"{symbol} Trade Analysis"),
            'performance_summary': Visualizer.plot_performance_summary(results, f"{symbol} Performance Summary"),
            'trade_distribution': Visualizer.plot_trade_distribution(results['trades'], f"{symbol} Trade Distribution")
        }
        
        return visualizations
    
    def save_results(self, output_dir="results"):
        """
        Save backtest results and trades to CSV files.
        
        Args:
            output_dir: Directory to save results
        """
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        for symbol, results in self.results.items():
            # Save trades
            if 'trades' in results and len(results['trades']) > 0:
                trades_file = os.path.join(output_dir, f"{symbol}_trades.csv")
                results['trades'].to_csv(trades_file, index=False)
                
            # Save equity curve
            if 'df' in results and 'equity' in results['df'].columns:
                equity_file = os.path.join(output_dir, f"{symbol}_equity.csv")
                results['df'][['time_utc', 'equity']].to_csv(equity_file, index=False)
                
            # Save performance metrics
            metrics = {k: v for k, v in results.items() if k not in ['df', 'trades', 'equity_curve']}
            metrics_file = os.path.join(output_dir, f"{symbol}_metrics.csv")
            pd.DataFrame([metrics]).to_csv(metrics_file, index=False)
    
    def run_multi_asset_backtest(self, symbols, start_date=None, end_date=None):
        """
        Run backtest on multiple assets and compare performance.
        
        Args:
            symbols: List of cryptocurrency symbols
            start_date: Start date for backtest
            end_date: End date for backtest
            
        Returns:
            Dictionary with aggregated results
        """
        all_results = {}
        performance_comparison = []
        
        for symbol in symbols:
            # Run individual backtest
            results = self.run_backtest(symbol, start_date, end_date)
            all_results[symbol] = results
            
            # Extract key metrics for comparison
            metrics = {
                'symbol': symbol,
                'total_return_pct': results['total_return_pct'],
                'annual_return_pct': results['annual_return_pct'],
                'max_drawdown_pct': results['max_drawdown_pct'],
                'win_rate': results['win_rate'],
                'profit_factor': results['profit_factor'],
                'sharpe_ratio': results['sharpe_ratio'],
                'total_trades': results['total_trades']
            }
            performance_comparison.append(metrics)
            
        # Create performance comparison DataFrame
        comparison_df = pd.DataFrame(performance_comparison)
        
        return {
            'individual_results': all_results,
            'performance_comparison': comparison_df
        }


# Example usage
if __name__ == "__main__":
    # Initialize trading system
    system = CryptoTradingSystem(data_dir="coinlion/data")
    
    # Set strategy with custom parameters
    system.set_strategy(
        strategy_type="macd",
        fast_period=12,
        slow_period=26,
        signal_period=9,
        signal_type="both"
    )
    
    # Set backtester with risk management parameters
    system.set_backtester(
        initial_capital=10000.0,
        position_size=0.1,
        commission_pct=0.001,
        take_profit_pct=0.05,
        stop_loss_pct=0.03,
        trailing_stop_pct=0.02
    )
    
    # Run backtest on BTC
    results_btc = system.run_backtest("btcusd", start_date="2020-01-01", end_date="2020-03-01")
    
    # Run backtest on ETH
    results_eth = system.run_backtest("ethusd", start_date="2020-01-01", end_date="2020-03-01")
    
    # Optimize strategy for BTC
    optimization_results = system.optimize_strategy("btcusd", start_date="2020-01-01", end_date="2020-03-01")
    best_params = optimization_results['best_parameters']
    print(f"Best MACD parameters for BTC: {best_params}")
    
    # Update strategy with optimized parameters
    system.set_strategy(
        strategy_type="macd",
        fast_period=best_params['fast'],
        slow_period=best_params['slow'],
        signal_period=best_params['signal']
    )
    
    # Run backtest with optimized parameters
    results_btc_optimized = system.run_backtest("btcusd", start_date="2020-01-01", end_date="2020-03-01")
    
    # Visualize results
    visualizations = system.visualize_results("btcusd")
    
    # Save results to CSV
    system.save_results(output_dir="results")
    
    # Run multi-asset backtest
    multi_results = system.run_multi_asset_backtest(
        symbols=["btcusd", "ethusd"],
        start_date="2020-01-01",
        end_date="2020-03-01"
    )
    
    # Print performance comparison
    print(multi_results['performance_comparison'])

FileNotFoundError: Data file for btcusd not found at coinlion/data\btcusd_10m.csv